# Augmented Test to SQL Grammar Parser
Uses an augmented context free grammar (CFG) to parse natural language queries into SQL queries to search the Air Traffic Information Systems (ATIS) database

In [16]:
import os
import nltk
from cryptography.fernet import Fernet
import copy
import datetime
import math
import re
import sys
import warnings
import wget
import sqlite3
import torch
import torch.nn as nn
import torchtext.legacy as tt
from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from cryptography.fernet import Fernet
from tqdm import tqdm

In [17]:
# Set random seeds
seed = 1234
torch.manual_seed(seed)
# Set timeout for executing SQL
TIMEOUT = 3 # seconds

# GPU check: Set runtime type to use GPU where available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (device)

cpu


In [18]:
## Download needed scripts and data
os.makedirs('data', exist_ok=True)
os.makedirs('scripts', exist_ok=True)
source_url = "https://raw.githubusercontent.com/nlp-course/data/master"

# Grammar to augment
if not os.path.isfile('data/grammar'):
  wget.download(f"{source_url}/ATIS/grammar_distrib4.crypt", out="data/")

  # Decrypt the grammar file
  key = b'bfksTY2BJ5VKKK9xZb1PDDLaGkdu7KCDFYfVePSEfGY='
  fernet = Fernet(key)
  with open('./data/grammar_distrib4.crypt', 'rb') as f:
    restored = Fernet(key).decrypt(f.read())
  with open('./data/grammar', 'wb') as f:
    f.write(restored)

# Download scripts and ATIS database
wget.download(f"{source_url}/scripts/trees/transform.py", out="scripts/")
wget.download(f"{source_url}/ATIS/atis_sqlite.db", out="data/")

100% [....................................................] 16404480 / 16404480

'data//atis_sqlite (1).db'

In [19]:
# Import downloaded scripts for parsing augmented grammars
sys.path.insert(1, './scripts')
import transform as xform

Load and preprocess the grammar

In [20]:
# Acquire the datasets - training, development, and test splits of the 
# ATIS queries and corresponding SQL queries
wget.download(f"{source_url}/ATIS/test_flightid.nl", out="data/")
wget.download(f"{source_url}/ATIS/test_flightid.sql", out="data/")
wget.download(f"{source_url}/ATIS/dev_flightid.nl", out="data/")
wget.download(f"{source_url}/ATIS/dev_flightid.sql", out="data/")
wget.download(f"{source_url}/ATIS/train_flightid.nl", out="data/")
wget.download(f"{source_url}/ATIS/train_flightid.sql", out="data/")

100% [......................................................] 2591248 / 2591248

'data//train_flightid (1).sql'

Use torchtext to process the data, with field SRC for the natural language questions and TGT for the SQL queries.

In [21]:
## Tokenizer
tokenizer = nltk.tokenize.RegexpTokenizer('\d+|st\.|[\w-]+|\$[\d\.]+|\S+')
def tokenize(string):
  return tokenizer.tokenize(string.lower())

In [22]:
SRC = tt.data.Field(include_lengths=True,         # include lengths
                    batch_first=False,            # batches will be max_len x batch_size
                    tokenize=tokenize,            # use our tokenizer
                   ) 
TGT = tt.data.Field(include_lengths=False,
                    batch_first=False,            # batches will be max_len x batch_size
                    tokenize=lambda x: x.split(), # use split to tokenize
                    init_token="<bos>",           # prepend <bos>
                    eos_token="<eos>")            # append <eos>
fields = [('src', SRC), ('tgt', TGT)]

In [23]:
 # Make splits for data
train_data, val_data, test_data = tt.datasets.TranslationDataset.splits(
    ('_flightid.nl', '_flightid.sql'), fields, path='./data/',
    train='train', validation='dev', test='test')

MIN_FREQ = 3
SRC.build_vocab(train_data.src, min_freq=MIN_FREQ)
TGT.build_vocab(train_data.tgt, min_freq=MIN_FREQ)

print (f"Size of English vocab: {len(SRC.vocab)}")
print (f"Most common English words: {SRC.vocab.freqs.most_common(10)}\n")

print (f"Size of SQL vocab: {len(TGT.vocab)}")
print (f"Most common SQL words: {TGT.vocab.freqs.most_common(10)}\n")

print (f"Index for start of sequence token: {TGT.vocab.stoi[TGT.init_token]}")
print (f"Index for end of sequence token: {TGT.vocab.stoi[TGT.eos_token]}")

Size of English vocab: 421
Most common English words: [('to', 3478), ('from', 3019), ('flights', 2094), ('the', 1550), ('on', 1230), ('me', 973), ('flight', 972), ('show', 845), ('what', 833), ('boston', 813)]

Size of SQL vocab: 392
Most common SQL words: [('=', 38876), ('AND', 36564), (',', 22772), ('airport_service', 8314), ('city', 8313), ('(', 6432), (')', 6432), ('flight_1.flight_id', 4536), ('flight', 4221), ('SELECT', 4178)]

Index for start of sequence token: 2
Index for end of sequence token: 3


Batch the data to facilitate processing on a GPU. sort_key function allows for sorting on length, to minimize the padding on the source side.

In [24]:
BATCH_SIZE = 16 # batch size for training/validation
TEST_BATCH_SIZE = 1 # batch size for test, we use 1 to make beam search implementation easier

train_iter, val_iter = tt.data.BucketIterator.splits((train_data, val_data),
                                                     batch_size=BATCH_SIZE, 
                                                     device=device,
                                                     repeat=False, 
                                                     sort_key=lambda x: len(x.src), 
                                                     sort_within_batch=True)
test_iter = tt.data.BucketIterator(test_data, 
                                   batch_size=TEST_BATCH_SIZE, 
                                   device=device,
                                   repeat=False, 
                                   sort=False, 
                                   train=False)

In [25]:
batch = next(iter(train_iter))
train_batch_text, train_batch_text_lengths = batch.src
train_batch_sql = batch.tgt

Set up a SQL database to test the parses correctly return the right database entries using sqlite3 module.

In [26]:
def execute_sql(sql):
  conn = sqlite3.connect('data/atis_sqlite.db')  # establish the DB based on the downloaded data
  c = conn.cursor()                              # build a "cursor"
  c.execute(sql)
  results = list(c.fetchall())
  c.close()
  conn.close()
  return results



<img src="https://github.com/nlp-course/data/raw/master/img/encoderdecoder_attn_1layer.png" alt="encoder-decoder-attn illustration" />

Implement the class `AttnEncoderDecoder` to convert natural language queries into SQL statements. The following methods are used to implement the Encoder-Decoder

* **Model**

    1. `__init__`: an initializer to create network modules.

    2. `forward`: given source word ids of size `(max_src_len, batch_size)`, source lengths of size `(batch_size)` and decoder input target word ids `(max_tgt_len, batch_size)`, returns logits `(max_tgt_len, batch_size, V_tgt)`. Implements two functions: `forward_encoder` and `forward_decoder`.

* **Optimization**

    3. `train_all`: compute loss on training data, compute gradients, and update model parameters to minimize the loss.

    4. `evaluate_ppl`: evaluate the current model's perplexity on a given dataset iterator, we use the perplexity value on the validation set to select the best model.

* **Decoding**

    5. `predict`: Generates the target sequence given a list of source tokens using beam search decoding. Assume batch size of 1 for simplicity 

In [27]:
#Implement global attention function
def attention(batched_Q, batched_K, batched_V, mask=None):
  """
  Performs the attention operation and returns the attention matrix
  `batched_A` and the context matrix `batched_C` using queries 
  `batched_Q`, keys `batched_K`, and values `batched_V`.
  
  Arguments:
      batched_Q: (q_len, bsz, D)
      batched_K: (k_len, bsz, D)
      batched_V: (k_len, bsz, D)
      mask: (bsz, q_len, k_len). An optional boolean mask *disallowing* 
            attentions where the mask value is *`False`*.
  Returns:
      batched_A: the normalized attention scores (bsz, q_len, k_len)
      batched_C: a tensor of size (q_len, bsz, D).
  """
  # Check sizes
  D = batched_Q.size(-1)
  bsz = batched_Q.size(1)
  q_len = batched_Q.size(0)
  k_len = batched_K.size(0)
  assert batched_K.size(-1) == D and batched_V.size(-1) == D
  assert batched_K.size(1) == bsz and batched_V.size(1) == bsz
  assert batched_V.size(0) == k_len
  if mask is not None:
    assert mask.size() == torch.Size([bsz, q_len, k_len])
  K = torch.transpose(torch.transpose(batched_K, 0, 2), 0, 1) # gives (bsz, D, k_len)
  Q = torch.transpose(batched_Q, 0, 1) # gives (bsz, q_len, D)
  QK = torch.bmm(Q, K)
  if mask != None:
    mask = mask == False # change to where mask does not exist
    QK = QK.masked_fill(mask, -math.inf) # fill in where there is no mask with - inf
    
  batched_A = torch.softmax(QK, dim = -1) # gives (bsz, q_len, k_len), only do masked fill if masked not none
  batched_C = torch.transpose(torch.bmm(batched_A, torch.transpose(batched_V, 0, 1)), 0, 1) # gives (q_len, bsz, D)
  # Verify that things sum up to one properly.
  assert torch.all(torch.isclose(batched_A.sum(-1), 
                                 torch.ones(bsz, q_len).to(device)))
  return batched_A, batched_C

In [28]:
class Beam():
  """
  Helper class for storing a hypothesis, its score and its decoder hidden state.
  """
  def __init__(self, decoder_state, tokens, score):
    self.decoder_state = decoder_state
    self.tokens = tokens
    self.score = score
        
class BeamSearcher():
  """
  Main class for beam search.
  """
  def __init__(self, model):
    self.model = model
    self.bos_id = model.bos_id
    self.eos_id = model.eos_id
    self.padding_id_src = model.padding_id_src
    self.V = model.V_tgt


  def beam_search(self, src, src_lengths, K, max_T):
    """
    Performs beam search decoding.
    Arguments:
        src: src batch of size (max_src_len, 1)
        src_lengths: src lengths of size (1)
        K: beam size
        max_T: max possible target length considered
    Returns:
        a list of token ids and a list of attentions
    """
    finished = []
    all_attns = []
    # Initialize the beam
    self.model.eval()
    # find memory bank, encoder final state, and initialize beams

    memory_bank, encoder_final_state = self.model.forward_encoder(src, src_lengths)
    init_beam = Beam(encoder_final_state, [torch.tensor([self.bos_id])], 0)
    beams = [init_beam]
    
    with torch.no_grad():
      for t in range(max_T): # main body of search over time steps
        
        # Expand each beam by all possible tokens y_{t+1}
        all_total_scores = []
        for beam in beams:
          y_1_to_t, score, decoder_state = beam.tokens, beam.score, beam.decoder_state
          y_t = y_1_to_t[-1]
          #TODO - finish the code below
          # Hint: you might want to use `model.forward_decoder_incrementally` with `normalize=True`
          
          src_mask = src.ne(self.padding_id_src)
          logits, decoder_state, attn = \
          self.model.forward_decoder_incrementally(decoder_state, 
                                                   y_t, 
                                                   memory_bank, 
                                                   src_mask, 
                                                   normalize = True)
          total_scores = logits + score
          all_total_scores.append(total_scores)
          all_attns.append(attn) # keep attentions for visualization
          beam.decoder_state = decoder_state # update decoder state in the beam
        all_total_scores = torch.stack(all_total_scores) # (K, V) when t>0, (1, V) when t=0

        # Find K best next beams
        # The code below has the same functionality as line 6-12, but is more efficient
        all_scores_flattened = all_total_scores.view(-1) # K*V when t>0, 1*V when t=0
        topk_scores, topk_ids = all_scores_flattened.topk(K, 0)
        beam_ids = topk_ids.div(self.V, rounding_mode='floor')
        next_tokens = topk_ids - beam_ids * self.V
        new_beams = []
        for k in range(K):
          beam_id = beam_ids[k]       # which beam it comes from
          y_t_plus_1 = next_tokens[k] # which y_{t+1}
          score = topk_scores[k]
          beam = beams[beam_id]
          decoder_state = beam.decoder_state
          y_1_to_t = beam.tokens
          
          new_beam = Beam(decoder_state, y_1_to_t + [y_t_plus_1], score)
          new_beams.append(new_beam)
        beams = new_beams
        # Set aside completed beams
        # move completed beams to `finished` (and remove them from `beams`)
        finished += [beam for beam in beams if beam.tokens[-1] == self.eos_id]
        new_beams = [beam for beam in beams if beam.tokens[-1] != self.eos_id]
        beams = new_beams
        
        # Break the loop if everything is completed
        if len(beams) == 0:
            break
            
    # Return the best hypothesis
    if len(finished) > 0:
      finished = sorted(finished, key=lambda beam: -beam.score)
      return finished[0].tokens, all_attns
    else: # when nothing is finished, return an unfinished hypothesis
        return beams[0].tokens

In [29]:
class AttnEncoderDecoder(nn.Module):
  def __init__(self, src_field, tgt_field, hidden_size=64, layers=3):
    """
    Initializer. Creates network modules and loss function.
    Arguments:
        src_field: src field
        tgt_field: tgt field
        hidden_size: hidden layer size of both encoder and decoder
        layers: number of layers of both encoder and decoder
    """
    super().__init__()
    self.src_field = src_field
    self.tgt_field = tgt_field
    
    # Keep the vocabulary sizes available
    self.V_src = len(src_field.vocab.itos)
    self.V_tgt = len(tgt_field.vocab.itos)
    
    # Get special word ids
    self.padding_id_src = src_field.vocab.stoi[src_field.pad_token]
    self.padding_id_tgt = tgt_field.vocab.stoi[tgt_field.pad_token]
    self.bos_id = tgt_field.vocab.stoi[tgt_field.init_token]
    self.eos_id = tgt_field.vocab.stoi[tgt_field.eos_token]

    # Keep hyper-parameters available
    self.embedding_size = hidden_size
    self.hidden_size = hidden_size
    self.layers = layers

    # Create essential modules
    self.word_embeddings_src = nn.Embedding(self.V_src, self.embedding_size)
    self.word_embeddings_tgt = nn.Embedding(self.V_tgt, self.embedding_size)

    # RNN cells
    self.encoder_rnn = nn.LSTM(
      input_size    = self.embedding_size,
      hidden_size   = hidden_size // 2, # to match decoder hidden size
      num_layers    = layers,
      bidirectional = True              # bidirectional encoder
    )
    self.decoder_rnn = nn.LSTM(
      input_size    = self.embedding_size,
      hidden_size   = hidden_size,
      num_layers    = layers,
      bidirectional = False             # unidirectional decoder
    )

    # Final projection layer
    self.hidden2output = nn.Linear(2*hidden_size, self.V_tgt) # project the concatenation to logits
   
    # Create loss function
    self.loss_function = nn.CrossEntropyLoss(reduction='sum', 
                                             ignore_index=self.padding_id_tgt)

  def forward_encoder(self, src, src_lengths):
    """
    Encodes source words `src`.
    Arguments:
        src: src batch of size (max_src_len, bsz)
        src_lengths: src lengths of size (bsz)
    Returns:
        memory_bank: a tensor of size (src_len, bsz, hidden_size)
        (final_state, context): `final_state` is a tuple (h, c) where h/c is of size 
                                (layers, bsz, hidden_size), and `context` is `None`. 
    """
    src = src.to(device)
    src_legnths = src_lengths.to(device)
    embeddings = self.word_embeddings_src(src).to(device)

    src_lengths = src_lengths.tolist()

    packed = pack(embeddings, src_lengths)
    o, (h, c) = self.encoder_rnn(packed)
    memory_bank, output_lens = unpack(o)
    
    h = h.reshape(self.layers, 2, -1, self.hidden_size//2)
    h = h.transpose(1,2)
    h = h.reshape(self.layers, -1, self.hidden_size)
    
    c = c.reshape(self.layers, 2, -1, self.hidden_size//2)
    c = c.transpose(1,2)
    c = c.reshape(self.layers, -1, self.hidden_size)
    
    final_state = (h, c)
    context = None
    
    return memory_bank, (final_state, context)

  def forward_decoder(self, encoder_final_state, tgt_in, memory_bank, src_mask):
    """
    Decodes based on encoder final state, memory bank, src_mask, and ground truth 
    target words.
    Arguments:
        encoder_final_state: (final_state, None) where final_state is the encoder
                             final state used to initialize decoder. None is the
                             initial context (there's no previous context at the
                             first step).
        tgt_in: a tensor of size (tgt_len, bsz)
        memory_bank: a tensor of size (src_len, bsz, hidden_size), encoder outputs 
                     at every position
        src_mask: a tensor of size (src_len, bsz): a boolean tensor, `False` where
                  src is padding (we disallow decoder to attend to those places).
    Returns:
        Logits of size (tgt_len, bsz, V_tgt) (before the softmax operation)
    """
    max_tgt_length = tgt_in.size(0)
    tgt_in = tgt_in.to(device)
    memory_bank = memory_bank.to(device)
    src_mask = src_mask.to(device)
   
    # Initialize decoder state, note that it's a tuple (state, context) here
    decoder_states = encoder_final_state
    
    all_logits = []
    for i in range(max_tgt_length):
      logits, decoder_states, attn = \
        self.forward_decoder_incrementally(decoder_states, 
                                           tgt_in[i], 
                                           memory_bank,
                                           src_mask,
                                           normalize=False)
      all_logits.append(logits)             # list of bsz, vocab_tgt
    all_logits = torch.stack(all_logits, 0) # tgt_len, bsz, vocab_tgt
    return all_logits

  def forward(self, src, src_lengths, tgt_in):
    """
    Performs forward computation, returns logits.
    Arguments:
        src: src batch of size (max_src_len, bsz)
        src_lengths: src lengths of size (bsz)
        tgt_in:  a tensor of size (tgt_len, bsz)
    """
    src_mask = src.ne(self.padding_id_src) # max_src_len, bsz
    # Forward encoder
    memory_bank, encoder_final_state = self.forward_encoder(src, src_lengths)
    # Forward decoder
    logits = self.forward_decoder(encoder_final_state, tgt_in, memory_bank, src_mask)
    
    return logits

  def forward_decoder_incrementally(self, prev_decoder_states, tgt_in_onestep, 
                                    memory_bank, src_mask,
                                    normalize=True):
    """
    Forward the decoder for a single step with token `tgt_in_onestep`.
    This function will be used both in `forward_decoder` and in beam search.
    Note that bsz can be greater than 1.
    Arguments:
        prev_decoder_states: a tuple (prev_decoder_state, prev_context). `prev_context`
                             is `None` for the first step
        tgt_in_onestep: a tensor of size (bsz), tokens at one step
        memory_bank: a tensor of size (src_len, bsz, hidden_size), encoder outputs 
                     at every position
        src_mask: a tensor of size (src_len, bsz): a boolean tensor, `False` where
                  src is padding (we disallow decoder to attend to those places).
        normalize: use log_softmax to normalize or not. Beam search needs to normalize,
                   while `forward_decoder` does not
    Returns:
        logits: log probabilities for `tgt_in_token` of size (bsz, V_tgt)
        decoder_states: (`decoder_state`, `context`) which will be used for the 
                        next incremental update
        attn: normalized attention scores at this step (bsz, src_len)
    """
    tgt_in_onestep = tgt_in_onestep.to(device)
    memory_bank = memory_bank.to(device)
    src_mask = src_mask.to(device)

    prev_decoder_state, prev_context = prev_decoder_states
    
    
    # get decoder output and state
    embeddings_tgt = self.word_embeddings_tgt(tgt_in_onestep).to(device)
    
    if prev_context is not None:
        embeddings_tgt = embeddings_tgt + prev_context 
        
    embeddings_tgt = embeddings_tgt.unsqueeze(0)
    
    decoder_output, decoder_state = self.decoder_rnn(embeddings_tgt, prev_decoder_state)
    
    # calculate attention and context using attention function
    src_mask = src_mask.transpose(0,1).unsqueeze(1)
    attn, context = attention(decoder_output, memory_bank, memory_bank, mask=src_mask)
    
    decoder_output = decoder_output.squeeze(0)
    attn = attn.squeeze(1)
    context = context.squeeze(0)
    
    # calculate logits of concatentation of decoder_output and attn projected to vocab size
    concatenated = torch.cat([context, decoder_output], dim=1)
    
    # get logits from forward function projected to vocabulary size
    logits = self.hidden2output(concatenated)
    decoder_states = (decoder_state, context)
    if normalize:
      logits = torch.log_softmax(logits, dim=-1)
    return logits, decoder_states, attn

  def evaluate_ppl(self, iterator):
    """Returns the model's perplexity on a given dataset `iterator`."""
    # Switch to eval mode
    self.eval()
    total_loss = 0
    total_words = 0
    for batch in iterator:
      # Input and target
      src, src_lengths = batch.src
      tgt = batch.tgt # max_length_sql, bsz
      tgt_in = tgt[:-1] # remove <eos> for decode input (y_0=<bos>, y_1, y_2)
      tgt_out = tgt[1:] # remove <bos> as target        (y_1, y_2, y_3=<eos>)
      # Forward to get logits
      logits = self.forward(src, src_lengths, tgt_in)
      # Compute cross entropy loss
      loss = self.loss_function(logits.view(-1, self.V_tgt), tgt_out.view(-1))
      total_loss += loss.item()
      total_words += tgt_out.ne(self.padding_id_tgt).float().sum().item()
    return math.exp(total_loss/total_words)

  def train_all(self, train_iter, val_iter, epochs=10, learning_rate=0.001):
    """Train the model."""
    # Switch the module to training mode
    self.train()
    # Use Adam to optimize the parameters
    optim = torch.optim.Adam(self.parameters(), lr=learning_rate)
    best_validation_ppl = float('inf')
    best_model = None
    # Run the optimization for multiple epochs
    for epoch in range(epochs): 
      total_words = 0
      total_loss = 0.0
      for batch in tqdm(train_iter):
        # Zero the parameter gradients
        self.zero_grad()
        # Input and target
        src, src_lengths = batch.src # text: max_src_length, bsz
        tgt = batch.tgt # max_tgt_length, bsz
        tgt_in = tgt[:-1] # Remove <eos> for decode input (y_0=<bos>, y_1, y_2)
        tgt_out = tgt[1:] # Remove <bos> as target        (y_1, y_2, y_3=<eos>)
        bsz = tgt.size(1)
        # Run forward pass and compute loss along the way.
        logits = self.forward(src, src_lengths, tgt_in)
        loss = self.loss_function(logits.view(-1, self.V_tgt), tgt_out.view(-1))
        # Training stats
        num_tgt_words = tgt_out.ne(self.padding_id_tgt).float().sum().item()
        total_words += num_tgt_words
        total_loss += loss.item()
        # Perform backpropagation
        loss.div(bsz).backward()
        optim.step()

      # Evaluate and track improvements on the validation dataset
      validation_ppl = self.evaluate_ppl(val_iter)
      self.train()
      if validation_ppl < best_validation_ppl:
        best_validation_ppl = validation_ppl
        self.best_model = copy.deepcopy(self.state_dict())
      epoch_loss = total_loss / total_words
     
  def predict(self, tokens, K, max_T):
    (src, src_lengths) = self.src_field.process([tokens])
    src = src.to(device)
    src_lengths = src_lengths.to(device)

    beam_searcher = BeamSearcher(model)
    
    result = beam_searcher.beam_search(src, src_lengths, K, max_T)
    sql_query = ""
    prediction = result[0]
    for i in prediction[1:-1]:
      string = self.tgt_field.vocab.itos[i]

      sql_query += " " + string 

    return sql_query

In [ ]:
EPOCHS = 16 # epochs; we recommend starting with a smaller number like 1
LEARNING_RATE = 1e-4 # learning rate

# Instantiate and train classifier
model = AttnEncoderDecoder(SRC, TGT,
  hidden_size    = 1024,
  layers         = 1,
).to(device)

model.train_all(train_iter, val_iter, epochs=EPOCHS, learning_rate=LEARNING_RATE)
model.load_state_dict(model.best_model)

# Evaluate model performance, the expected value should be < 1.2
print (f'Validation perplexity: {model.evaluate_ppl(val_iter):.3f}')